In [ ]:
%config InlineBackend.figure_format = 'retina'

from jax import debug
from jax import random
from numpyro.infer import NUTS, MCMC
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np
import numpyro
import numpyro.distributions as dist
import pandas as pd
import seaborn as sns

# plotting defaults
plt.style.use('fivethirtyeight')
plt.rcParams['axes.facecolor'] = 'white'
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.bottom'] = False
sns.set_palette("tab10")

# hyper parameters
SEED = 42
RNG = np.random.default_rng(SEED)
BUFFER = 100
M = 200
TUNING_DRAWS = 1000
DRAWS = 1000
CHAINS = 1
U_SIGMA = 150

RANDOM_SEED = 17

def get_oven_data(**kwargs):
    '''Get a (individual_count, trap_count) array of detections.'''

    # coordinates for each trap
    ovenbird_trap = pd.read_csv('ovenbirdtrap.txt', delimiter=' ')

    # information about each trap
    trap_coords = ovenbird_trap[['x', 'y']].to_numpy()

    # ovenbird capture history
    oven_ch = pd.read_csv('ovenbirdcapt.txt', delimiter=' ')

    # create a unique bird/year identifier for each individual
    oven_ch['ID'] = oven_ch.groupby(['Year','Band']).ngroup()
    occasion_count = oven_ch.Day.max()

    # merge the datasets, making sure that traps with no detections are included
    ovenbird = (
        ovenbird_trap.merge(oven_ch[['ID', 'Net', 'Day']], how='left')
          [['ID', 'Day', 'Net', 'x', 'y']]
          .sort_values('ID')
          .reset_index(drop=True)
    )

    # count the number of detections per individual per trap
    detection_counts = pd.crosstab(ovenbird.ID, ovenbird.Net, dropna=False)

    # remove the ghost nan individual
    detection_counts = detection_counts.loc[~detection_counts.index.isna()]

    capture_histories = detection_counts.to_numpy()

    return {
        'capture_histories': capture_histories,
        'trap_coords': trap_coords,
        'occasion_count': occasion_count,
        'buffer': kwargs['buffer'],
        'superpop_size': kwargs['M']
    }

oven = get_oven_data(buffer=BUFFER, M=M)

In [ ]:
# coordinates for each trap
ovenbird_trap = pd.read_csv('ovenbirdtrap.txt', delimiter=' ')

# information about each trap
trap_coords = ovenbird_trap[['x', 'y']].to_numpy()

# ovenbird capture history
oven_ch = pd.read_csv('ovenbirdcapt.txt', delimiter=' ')

oven_ch

,Year,Band,Day,Net,Sex
0,2005,1911.52067,1,2,F
1,2005,1911.52069,1,15,F
2,2005,1911.52069,9,20,F
3,2005,1911.52070,1,14,M
4,2005,1911.52070,1,15,M
...,...,...,...,...,...
217,2009,1911.52291,10,9,F
218,2009,1911.52296,4,31,F
219,2009,1911.52296,5,30,F
220,2009,1911.52297,4,32,M


In [ ]:
df = oven_ch

# Get unique values for each dimension
bands = df['Band'].unique()
nets = df['Net'].unique()
days = df['Day'].unique()
years = df['Year'].unique()

# Create mapping dictionaries for fast lookup
band_to_idx = {band: i for i, band in enumerate(bands)}
net_to_idx = {net: i for i, net in enumerate(nets)}
day_to_idx = {day: i for i, day in enumerate(days)}
year_to_idx = {year: i for i, year in enumerate(years)}

# Initialize 4D array
y = np.zeros((len(bands), len(nets), len(days), len(years)), dtype=np.int8)

# Vectorized indexing
band_indices = df['Band'].map(band_to_idx).values
net_indices = df['Net'].map(net_to_idx).values
day_indices = df['Day'].map(day_to_idx).values
year_indices = df['Year'].map(year_to_idx).values

y[band_indices, net_indices, day_indices, year_indices] = 1

(70, 43, 11, 5)